In [22]:
import pandas as pd
import numpy as np

In [23]:
# Importing final dataset used by topic clustering model 

clean = pd.read_csv('clean.csv').drop(columns=['Unnamed: 0'])

In [3]:
clean.shape

(4328, 7)

In [4]:
clean.head()

,outlet,url,title,authors,publish_date,text,keywords
0,cbc,https://www.cbc.ca/news/covid-19,CBC News | Information about COVID-19 in Canada,'Darren Bernhardt',NaN,Sylvan Lake to clamp down on beach crowds with...,[]
1,cbc,https://www.cbc.ca/news/local,CBC News,'Andrew Kurjata',NaN,What you need to know about COVID-19 in Ottawa...,[]
2,cbc,https://www.cbc.ca/news/politics,Politics,'John Paul Tasker',NaN,WE Charity contract could have been worth up t...,[]
3,cbc,https://www.cbc.ca/news/indigenous,Indigenous,'Jessica Deer',NaN,Manitoba judge points to systemic issues in In...,[]
4,cbc,https://www.cbc.ca/news/business,Business,'Pete Evans',NaN,Twitter says about 130 accounts were targeted ...,[]


There are 21 unique news outlets in this dataset.

In [5]:
clean['outlet'].unique()

array(['cbc', 'ctvnews', 'nationalpost', 'torontosun', 'thestar', 'cp24',
       'mapleridgenews', 'tricitynews', 'langleyadvancetimes', 'abbynews',
       'theprogress', 'northdeltareporter', 'surreynowleader',
       'vancouverobserver', 'vancourier', 'nsnews', 'richmond-news',
       'burnabynow', 'newwestrecord', 'bowenislandundercurrent',
       'The Record'], dtype=object)

**Options for location-based coordinates for Demo Day:**

1. Count number of locations in the text and find coordinates of most frequent 
    - spacy API
    
2. Use location of outlet (21 unique locations) 

**Pipeline:**

- For all text that have GPE (location) items,
        - Count occurrences of location names in text
        - Select most frequently mentioned location  **(1A)**

- For text without GPE item,
        - Select location of news outlet  **(1B)**
        
        
- Find coordinates of location **(2)**
        - use mapbox API and forward geocoding (https://docs.mapbox.com/api/search/#forward-geocoding)
        - Add noise to lat and long coordinates 

## (1) Determining location of article

In [6]:
import spacy

### Create dictionary of news outlets and location

In [7]:
def find_location(text, model):
    
    doc = model(text)
    final_loc = ''
    locations = []
    
    for ent in doc.ents:
        if (ent.label_ == 'GPE'):
            locations.append(ent.text)
            #print(ent.text, ent.start_char, ent.end_char, ent.label_)
    
    if len(locations) > 0:
        loc_counts = pd.DataFrame([[x, locations.count(x)] for x in set(locations)])
        loc_counts.columns = ['location', 'count']
        loc_counts = loc_counts.sort_values(by=['count'], ascending=False)
    
        final_loc = loc_counts.iloc[0, 0]
        
    return final_loc   

### Creating new .csv of articles with locations

In [94]:
new_clean = clean

In [9]:
# Model to use for location 

nlp = spacy.load("en_core_web_sm")

In [95]:
count_spacy = 0
count_outlet = 0

In [101]:
for i in range(len(new_clean)):
    
    # Finding location of article
    print('Row: ' + str(i))
    text_loc = find_location(new_clean.loc[i, 'text'], nlp)
    
    if text_loc=='Surrey':
        text_loc = 'Surrey, British Columbia'
    elif text_loc=='B.C.':
        text_loc = 'British Columbia'
        
    print('Location from spacy: ' + text_loc)
    
    if text_loc != '': 
        try: 
            new_clean.loc[i, 'location'] = text_loc        # (1A)
            loc = gl.geocode(text_loc)
            lat, long = loc.latitude, loc.longitude
            
            print('Coordinates from geopy: [' + str(lat) + ', ' + str(long) + ']' )
            
            new_clean.loc[i, 'lat'] = lat
            new_clean.loc[i, 'long'] = long
                
            count_spacy +=1
            
        except:
            print("Error with .")
            new_clean.loc[i, 'lat'] = 0.0
            new_clean.loc[i, 'long'] = 0.0
            
    else:
        try:
            new_clean.loc[i, 'location'] = loc_dict[new_clean.loc[i, 'outlet']]      # (1B)
            new_clean.loc[i, 'lat'] = lat_dict[new_clean.loc[i, 'outlet']] 
            new_clean.loc[i, 'long'] = long_dict[new_clean.loc[i, 'outlet']]  
            count_outlet += 1
        except:
            print("Error calling from dictionaries.")

Row: 0
Location from spacy: Alberta
Coordinates from geopy: [55.001251, -115.002136]
Row: 1
Location from spacy: Ottawa
Coordinates from geopy: [45.421106, -75.690308]
Row: 2
Location from spacy: 
Row: 3
Location from spacy: Manitoba
Coordinates from geopy: [55.001251, -97.001038]
Row: 4
Location from spacy: 
Row: 5
Location from spacy: 
Row: 6
Location from spacy: 
Row: 7
Location from spacy: N.S.
Coordinates from geopy: [24.4349856, 105.5550331]
Row: 8
Location from spacy: 
Row: 9
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 10
Location from spacy: Greece
Coordinates from geopy: [38.9953683, 21.9877132]
Row: 11
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 12
Location from spacy: 
Row: 13
Location from spacy: Moscow
Coordinates from geopy: [55.7504461, 37.6174943]
Row: 14
Location from spacy: Jordan
Coordinates from geopy: [31.1667049, 36.941628]
Row: 15
Location from spacy: Canada
Coordinates from geopy: [6

Coordinates from geopy: [55.001251, -125.002441]
Row: 101
Location from spacy: Italy
Coordinates from geopy: [42.6384261, 12.674297]
Row: 102
Location from spacy: Toronto
Coordinates from geopy: [43.6534817, -79.3839347]
Row: 103
Location from spacy: Orchestre Métropolitain
Error with .
Row: 104
Location from spacy: Montreal
Coordinates from geopy: [45.4972159, -73.6103642]
Row: 105
Location from spacy: New Orleans
Coordinates from geopy: [29.9499323, -90.0701156]
Row: 106
Location from spacy: Victoria
Coordinates from geopy: [-36.5986096, 144.6780052]
Row: 107
Location from spacy: Toronto
Coordinates from geopy: [43.6534817, -79.3839347]
Row: 108
Location from spacy: Cambodia
Coordinates from geopy: [13.5066394, 104.869423]
Row: 109
Location from spacy: Xoxox
Error with .
Row: 110
Location from spacy: Alessia
Coordinates from geopy: [40.6855514, 14.7325899]
Row: 111
Location from spacy: 
Row: 112
Location from spacy: Weeknd
Coordinates from geopy: [10.4273236, -85.0943199]
Row: 113
Lo

Coordinates from geopy: [45.421106, -75.690308]
Row: 199
Location from spacy: 
Row: 200
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 201
Location from spacy: Ottawa
Coordinates from geopy: [45.421106, -75.690308]
Row: 202
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 203
Location from spacy: 
Row: 204
Location from spacy: Alberta
Coordinates from geopy: [55.001251, -115.002136]
Row: 205
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 206
Location from spacy: British Columbia
Coordinates from geopy: [55.001251, -125.002441]
Row: 207
Location from spacy: 
Row: 208
Location from spacy: McKnight
Coordinates from geopy: [40.5550681, -80.0364448]
Row: 209
Location from spacy: Edmonton
Coordinates from geopy: [53.535411, -113.507996]
Row: 210
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 211
Location from spacy: Edmonton
Coordinates from geopy:

Coordinates from geopy: [55.001251, -125.002441]
Row: 301
Location from spacy: Vancouver
Coordinates from geopy: [49.2608724, -123.1139529]
Row: 302
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 303
Location from spacy: Vancouver
Coordinates from geopy: [49.2608724, -123.1139529]
Row: 304
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 305
Location from spacy: British Columbia
Coordinates from geopy: [55.001251, -125.002441]
Row: 306
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 307
Location from spacy: Langley
Coordinates from geopy: [48.3604648, 6.3258262]
Row: 308
Location from spacy: 
Row: 309
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 310
Location from spacy: Vancouver
Coordinates from geopy: [49.2608724, -123.1139529]
Row: 311
Location from spacy: South Burnaby
Coordinates from geopy: [49.2433804, -122.9725459]
Row: 312


Coordinates from geopy: [55.001251, -115.002136]
Row: 410
Location from spacy: Wisconsin
Coordinates from geopy: [44.4308975, -89.6884637]
Row: 411
Location from spacy: U.S.
Coordinates from geopy: [45.1895845, -88.73215845]
Row: 412
Location from spacy: Edinburgh
Coordinates from geopy: [55.9533456, -3.1883749]
Row: 413
Location from spacy: U.K.
Coordinates from geopy: [54.7023545, -3.2765753]
Row: 414
Location from spacy: Selma
Coordinates from geopy: [32.4078632, -87.0207473]
Row: 415
Location from spacy: Traore
Coordinates from geopy: [16.068147, 0.0648645]
Row: 416
Location from spacy: China
Coordinates from geopy: [35.000074, 104.999927]
Row: 417
Location from spacy: Calgary
Coordinates from geopy: [51.0534234, -114.0625892]
Row: 418
Location from spacy: Calgary
Coordinates from geopy: [51.0534234, -114.0625892]
Row: 419
Location from spacy: Alberta
Coordinates from geopy: [55.001251, -115.002136]
Row: 420
Location from spacy: Alberta
Coordinates from geopy: [55.001251, -115.0021

...

Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3866
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3867
Location from spacy: Uptown
Coordinates from geopy: [29.9364059, -90.101181]
Row: 3868
Location from spacy: Vancouver
Coordinates from geopy: [49.2608724, -123.1139529]
Row: 3869
Location from spacy: mem.com
Coordinates from geopy: [38.78940245, -9.352990923077778]
Row: 3870
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3871
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3872
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3873
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 3874
Location from spacy: 
Row: 3875
Location from spacy: Sapperton
Coordinates from geopy: [49.224545, -122.8895213]
Row: 3876
Location from spacy: British Columbia
Coordinates from geop

Coordinates from geopy: [49.12120935, -56.69629621274099]
Row: 3963
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3964
Location from spacy: British Columbia
Coordinates from geopy: [55.001251, -125.002441]
Row: 3965
Location from spacy: Burnaby
Coordinates from geopy: [49.2433804, -122.9725459]
Row: 3966
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3967
Location from spacy: Coquitlam
Coordinates from geopy: [49.2842958, -122.793281]
Row: 3968
Location from spacy: Sapperton
Coordinates from geopy: [49.224545, -122.8895213]
Row: 3969
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3970
Location from spacy: U.S.
Coordinates from geopy: [45.1895845, -88.73215845]
Row: 3971
Location from spacy: New Westminster
Coordinates from geopy: [49.2067726, -122.9108818]
Row: 3972
Location from spacy: New Westminster-Burnaby
Coordinates from geopy: [49.2312928, -1

Coordinates from geopy: [-41.5000831, 172.8344077]
Row: 4072
Location from spacy: BC
Coordinates from geopy: [55.001251, -125.002441]
Row: 4073
Location from spacy: 
Row: 4074
Location from spacy: Bastin
Coordinates from geopy: [48.5608587, 18.236212]
Row: 4075
Location from spacy: Vancouver
Coordinates from geopy: [49.2608724, -123.1139529]
Row: 4076
Location from spacy: 
Row: 4077
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 4078
Location from spacy: Bronwyn
Coordinates from geopy: [-33.9406024, 25.486865]
Row: 4079
Location from spacy: Halti
Coordinates from geopy: [69.3080614, 21.263321]
Row: 4080
Location from spacy: 
Row: 4081
Location from spacy: turkey
Coordinates from geopy: [38.9597594, 34.9249653]
Row: 4082
Location from spacy: 
Row: 4083
Location from spacy: British Columbia
Coordinates from geopy: [55.001251, -125.002441]
Row: 4084
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 4085
Location from s

Coordinates from geopy: [55.001251, -115.002136]
Row: 4172
Location from spacy: Daadab
Coordinates from geopy: [-0.7732949, 38.357783]
Row: 4173
Location from spacy: 
Row: 4174
Location from spacy: Nova Scotia
Coordinates from geopy: [45.1960403, -63.1653789]
Row: 4175
Location from spacy: Dorton
Coordinates from geopy: [37.2767726, -82.5790436]
Row: 4176
Location from spacy: New Brunswick
Coordinates from geopy: [46.500283, -66.750183]
Row: 4177
Location from spacy: Toronto
Coordinates from geopy: [43.6534817, -79.3839347]
Row: 4178
Location from spacy: Toronto
Coordinates from geopy: [43.6534817, -79.3839347]
Row: 4179
Location from spacy: Ontario
Coordinates from geopy: [50.000678, -86.000977]
Row: 4180
Location from spacy: London
Coordinates from geopy: [51.5073219, -0.1276474]
Row: 4181
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.9917071]
Row: 4182
Location from spacy: 
Row: 4183
Location from spacy: Canada
Coordinates from geopy: [61.0666922, -107.991707

Coordinates from geopy: [22.2793278, 114.1628131]
Row: 4272
Location from spacy: U.S.
Coordinates from geopy: [45.1895845, -88.73215845]
Row: 4273
Location from spacy: U.S.
Coordinates from geopy: [45.1895845, -88.73215845]
Row: 4274
Location from spacy: Los Angeles
Coordinates from geopy: [34.0536909, -118.2427666]
Row: 4275
Location from spacy: St. Louis
Coordinates from geopy: [38.6268039, -90.1994097]
Row: 4276
Location from spacy: U.S.
Coordinates from geopy: [45.1895845, -88.73215845]
Row: 4277
Location from spacy: U.S.
Coordinates from geopy: [45.1895845, -88.73215845]
Row: 4278
Location from spacy: Mississippi
Coordinates from geopy: [32.9715645, -89.7348497]
Row: 4279
Location from spacy: Michigan
Coordinates from geopy: [43.6211955, -84.6824346]
Row: 4280
Location from spacy: Alaska
Coordinates from geopy: [64.4459613, -149.680909]
Row: 4281
Location from spacy: U.S.
Coordinates from geopy: [45.1895845, -88.73215845]
Row: 4282
Location from spacy: Rayong
Coordinates from geop

### Exploring dataset with locations (new_clean.csv)

In [102]:
count_outlet

684

In [103]:
count_spacy

3585

In [104]:
new_clean.head()

,outlet,url,title,authors,publish_date,text,keywords,location,lat,long
0,cbc,https://www.cbc.ca/news/covid-19,CBC News | Information about COVID-19 in Canada,'Darren Bernhardt',NaN,Sylvan Lake to clamp down on beach crowds with...,[],Alberta,55.001251,-115.002136
1,cbc,https://www.cbc.ca/news/local,CBC News,'Andrew Kurjata',NaN,What you need to know about COVID-19 in Ottawa...,[],Ottawa,45.421106,-75.690308
2,cbc,https://www.cbc.ca/news/politics,Politics,'John Paul Tasker',NaN,WE Charity contract could have been worth up t...,[],Toronto,45.518055,-73.550930
3,cbc,https://www.cbc.ca/news/indigenous,Indigenous,'Jessica Deer',NaN,Manitoba judge points to systemic issues in In...,[],Manitoba,55.001251,-97.001038
4,cbc,https://www.cbc.ca/news/business,Business,'Pete Evans',NaN,Twitter says about 130 accounts were targeted ...,[],Toronto,45.518055,-73.550930


In [105]:
new_clean['location'].value_counts()

Vancouver              417
British Columbia       319
Toronto                285
Burnaby                284
Canada                 283
                      ... 
Hydra Perfection         1
C.Dir                    1
hollowaypainting.ca      1
Whonnock                 1
Liberty Village and      1
Name: location, Length: 896, dtype: int64

**Notes:**
- Change "Surrey" to "Surrey, British Columbia"
- Change "B.C." to "British Columbia"

## Exploring geo.csv (locations of outlets) 

In [2]:
geo = pd.read_csv('geo.csv')

In [3]:
clean.columns

NameError: name 'clean' is not defined

In [4]:
geo.columns = ['outlet', 'url', 'title', 'authors', 'publish_date', 'text', 'keywords', 'scope', 'location', 'latitude', 'longitude']

In [5]:
geo.head()

,outlet,url,title,authors,publish_date,text,keywords,scope,location,latitude,longitude
0,ctvnews,https://www.ctvnews.ca/health/coronavirus/trac...,Tracking every case of COVID-19 in Canada,NaN,2020-03-13 14:17:00-04:00,July 17 – Ontario health officials recorded 11...,[],national,Ottawa,-75.697472,45.421861
1,ctvnews,https://www.ctvnews.ca/politics/liberals-revis...,"Liberals revise COVID-19 wage subsidy, ease el...",NaN,2020-07-17 10:22:00-04:00,OTTAWA -- Finance Minister Bill Morneau says t...,[],national,Ottawa,-75.697472,45.421861
2,ctvnews,https://www.ctvnews.ca/health/coronavirus/new-...,New Normal: Casinos betting on temperature che...,NaN,2020-07-17 12:03:00-04:00,TORONTO -- Canadian gamblers are trading in ca...,[],national,Ottawa,-75.697472,45.421861
3,ctvnews,https://www.ctvnews.ca/health/coronavirus/it-s...,'It's amazing': Toronto man recovering after 1...,NaN,2020-07-15 22:00:00-04:00,"TORONTO -- For months, Bruno Iozzo’s family wa...",[],national,Ottawa,-75.697472,45.421861
4,ctvnews,https://www.ctvnews.ca/health/coronavirus/ariz...,Arizona man in a coma from coronavirus wakes u...,'Madeline Holcombe',2020-07-16 08:15:00-04:00,"For Eddie Case, recovering from COVID-19 comes...",[],national,Ottawa,-75.697472,45.421861


### Creates dictionaries from scope and location of each outlet

In [6]:
group = geo.groupby('outlet')

In [7]:
# Not all 

scope_dict = group.apply(lambda x: x['scope'].unique()).apply(pd.Series).loc[:, 0].to_dict()

In [28]:
loc_dict = group.apply(lambda x: x['location'].unique()).apply(pd.Series).loc[:, 0].to_dict()

In [29]:
lat_dict = group.apply(lambda x: x['latitude'].unique()).apply(pd.Series).loc[:, 0].to_dict()

In [30]:
long_dict = group.apply(lambda x: x['longitude'].unique()).apply(pd.Series).loc[:, 0].to_dict()

In [31]:
loc_dict['cbc'] = 'Toronto'
loc_dict['tricitynews'] = 'Port Coquitlam'
loc_dict['vancouverobserver'] = 'Vancouver'
loc_dict['vancourier'] = 'Vancouver'
loc_dict['richmond-news'] = 'Richmond'
loc_dict['burnabynow'] = 'Burnaby'
loc_dict['newwestrecord'] = 'Burnaby'
loc_dict['bowenislandundercurrent'] = 'Bowen Island'

In [32]:
lat_dict['cbc'] = 45.518055
lat_dict['tricitynews'] = 49.247012
lat_dict['vancouverobserver'] = -123.10096000000001
lat_dict['vancourier'] = 49.266922
lat_dict['richmond-news'] = 49.174152
lat_dict['burnabynow'] = 49.252834
lat_dict['newwestrecord'] = 49.252857
lat_dict['bowenislandundercurrent'] = 49.379632

In [33]:
long_dict['cbc'] = -73.550930
long_dict['tricitynews'] = -122.760220
long_dict['vancouverobserver'] = 49.318767
long_dict['vancourier'] = -123.110863
long_dict['richmond-news'] = -123.119285
long_dict['burnabynow'] = -122.917969
long_dict['newwestrecord'] = -122.916369
long_dict['bowenislandundercurrent'] = -123.338253

In [34]:
out_lat = pd.DataFrame(lat_dict.items(), columns=['outlet', 'lat'])
out_long = pd.DataFrame(long_dict.items(), columns=['outlet', 'long'])

In [37]:
out_location = pd.DataFrame(loc_dict.items(), columns=['outlet', 'location'])
out_location = pd.merge(out_location, out_lat, how='inner', on=['outlet'])
out_location = pd.merge(out_location, out_long, how='inner', on=['outlet'])

In [39]:
out_location.to_csv('outlet_locations.csv')

### Exploring geopy API for coordinates

In [29]:
from geopy.geocoders import Nominatim

gl = Nominatim(user_agent='newsworthy_ml')

In [35]:
loc = gl.geocode('Kanye West')

In [36]:
loc.address

'Kanye, Southern District, PLOT, Botswana'

In [37]:
loc.latitude

-24.9766112

In [38]:
loc.longitude

25.3358859

## Combining with topic modelled articles

In [106]:
final = pd.read_csv('complete.csv')

In [107]:
final = final.drop(columns='Unnamed: 0')

In [108]:
new_clean2 = new_clean

In [109]:
final.columns

Index(['url', 'num_articles_per_topic', 'topic_resid', 'category',
       'category_probability', 'topic_num', 'topics', 'outlet', 'title',
       'authors', 'publish_date', 'text', 'keywords', 'word_count',
       'article_resid'],
      dtype='object')

In [110]:
new_clean2.columns

Index(['outlet', 'url', 'title', 'authors', 'publish_date', 'text', 'keywords',
       'location', 'lat', 'long'],
      dtype='object')

In [111]:
new_clean2[['url', 'location', 'lat', 'long']]

,url,location,lat,long
0,https://www.cbc.ca/news/covid-19,Alberta,55.001251,-115.002136
1,https://www.cbc.ca/news/local,Ottawa,45.421106,-75.690308
2,https://www.cbc.ca/news/politics,Toronto,45.518055,-73.550930
3,https://www.cbc.ca/news/indigenous,Manitoba,55.001251,-97.001038
4,https://www.cbc.ca/news/business,Toronto,45.518055,-73.550930
...,...,...,...,...
4323,https://www.therecord.com/opinion/columnists/2...,Toronto,43.653482,-79.383935
4324,https://www.therecord.com/opinion/columnists/2...,U.S.,45.189585,-88.732158
4325,https://www.therecord.com/opinion/contributors...,America,51.447704,5.966069
4326,https://www.therecord.com/opinion/contributors...,Ottawa,45.421106,-75.690308


In [112]:
new_clean2 = new_clean2[['url', 'location', 'lat', 'long']]

In [113]:
final_df = pd.merge(final, new_clean2, how='inner', on=['url'])

In [114]:
final_df.head()

,url,num_articles_per_topic,topic_resid,category,category_probability,topic_num,topics,outlet,title,authors,publish_date,text,keywords,word_count,article_resid,location,lat,long
0,https://www.cbc.ca/parents/learning/view/covid...,222,0.959933,lifestyle,86.343329,0,famili just time peopl year home life day,cbc,I Don’t Know If My Kids Will Ever See Their Gr...,'Jennifer Power Scott',2020-07-07,On a sticky New Brunswick afternoon — the kind...,[],1120,0.932753,Newfoundland,49.121209,-56.696296
1,https://www.thestar.com/life/health_wellness/2...,222,0.959933,lifestyle,86.343329,0,famili just time peopl year home life day,thestar,Why your workout may not be helping you lose w...,'Christine Sismondo',2020-01-06,This is the first in a week-long series on die...,[],954,0.968334,Toronto,43.653482,-79.383935
2,https://www.thestar.com/life/health_wellness/2...,222,0.959933,lifestyle,86.343329,0,famili just time peopl year home life day,thestar,She lost 100 pounds and became an Instagram se...,'Jen Kirsch',2020-01-09,This story is part of a week-long series on di...,[],1131,0.917396,Landry,45.570000,6.739910
3,https://www.thestar.com/life/relationships/adv...,222,0.959933,lifestyle,86.343329,0,famili just time peopl year home life day,thestar,How do I stop my adult children from being so ...,'Ellieadvice Columnisttue.',2020-07-14,"Q: I’m a male, 45, doing well professionally, ...",[],600,0.928283,Toronto,-79.540139,43.779361
4,https://www.ctvnews.ca/entertainment/magic-sch...,222,0.959933,lifestyle,86.343329,0,famili just time peopl year home life day,ctvnews,'Magic School Bus' author Joanna Cole dies at ...,NaN,2020-07-16 07:32:00-04:00,"NEW YORK -- Author Joanna Cole, whose ""Magic S...",[],374,0.982396,Sioux City,42.496682,-96.405878


In [115]:
final_df.shape

(1786, 18)

In [116]:
final_df.to_csv('final_df.csv')